<a href="https://colab.research.google.com/github/maxnaxmi/Python/blob/main/%E5%B1%A4%E5%88%A5%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AA%E3%83%B3%E3%82%B0%EF%BC%88spark%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# spark設定


In [22]:
# ColabにSparkとJavaをインストール
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [23]:
# Colab環境でPysparkを実行できるようにする環境パスを設定
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [24]:
# ローカルのSparkセッションを実行して、インストールをテスト
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [25]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [34]:
spark

## 1.層別サンプリング

sampleByサンプル　

https://stackoverflow.com/questions/47637760/stratified-sampling-with-pyspark


In [ ]:
# inputデータ作成
df = sc.parallelize([ (1,"A",282),(1,"B",179),(2,"B",178),(3,"C",191),(3,"A",828),(1,"A",158),(2,"B",231),(2,"A",199),(1,"B",385),(3,"C",128),
                    (1,"A",111),(1,"B",234),(2,"B",154),(3,"C",354),(3,"A",411),(1,"A",156),(2,"B",144),(2,"A",274),(1,"B",598),(3,"C",106),
                    (1,"A",421),(1,"A",224),(1,"A",400),(1,"A",237),(1,"A",256),(1,"A",384),(1,"A",250),(1,"A",421),(1,"A",508),(1,"A",614),
                    (1,"A",422),(1,"A",264),(1,"A",410),(1,"A",137),(1,"A",256),(1,"A",381),(1,"A",251),(1,"A",340),(1,"A",501),(1,"A",114),
                    (1,"A",423),(1,"A",614),(1,"A",420),(1,"A",437),(1,"A",257),(1,"A",382),(1,"A",252),(1,"A",480),(1,"A",502),(1,"A",214),
                    (1,"A",424),(1,"A",234),(1,"A",430),(1,"A",637),(1,"A",258),(1,"A",383),(1,"A",253),(1,"A",580),(1,"A",503),(1,"A",314),
                    (1,"A",425),(1,"A",224),(1,"A",440),(1,"A",238),(1,"A",259),(1,"A",384),(1,"A",254),(1,"A",680),(1,"A",504),(1,"A",414),
                    (1,"A",426),(1,"A",214),(1,"A",450),(1,"A",239),(1,"A",210),(1,"A",385),(1,"A",255),(1,"A",780),(1,"A",505),(1,"A",514),
                    (1,"A",416),(1,"A",221),(1,"A",399),(1,"A",202),(1,"A",362),(1,"A",341),(1,"A",202),(1,"A",810),(1,"A",581),(1,"A",124),
                    (1,"B",710),(1,"B",489),(1,"B",321),(1,"B",221),(1,"B",285),(1,"B",290),(1,"B",210),(1,"B",316),(1,"B",602),(1,"B",149),
                    (1,"A",631),(2,"C",143),(2,"A",154),(3,"C",476),(3,"A",832),(1,"C",115),(2,"B",168),(2,"A",284),(1,"B",509),(3,"C",115),
                    (1,"A",631),(1,"B",143),(2,"B",154),(1,"C",416),(3,"A",132),(2,"C",125),(2,"B",161),(3,"A",214),(1,"B",519),(3,"C",212),
                    (1,"A",621),(3,"C",143),(2,"C",154),(1,"A",425),(1,"A",232),(2,"C",131),(1,"B",162),(2,"A",224),(2,"B",529),(3,"A",314),
                    (1,"A",611),(1,"C",143),(2,"A",154),(1,"B",436),(1,"A",332),(2,"C",142),(1,"B",163),(2,"A",224),(2,"B",539),(3,"A",434),
                    (1,"A",691),(2,"C",143),(2,"B",254),(1,"C",446),(1,"A",432),(2,"C",153),(1,"B",164),(2,"A",234),(2,"B",549),(3,"A",424),
                    (1,"A",321),(3,"C",143),(2,"C",354),(1,"B",456),(1,"A",532),(2,"C",214),(1,"B",165),(2,"A",354),(2,"B",559),(3,"A",435),
                    (1,"A",231),(1,"C",143),(2,"A",454),(1,"A",466),(1,"A",632),(2,"C",315),(1,"B",166),(2,"A",464),(2,"B",569),(3,"A",446),
                    (1,"B",531),(2,"C",143),(2,"B",554),(1,"B",416),(1,"B",732),(2,"C",416),(1,"B",167),(2,"A",584),(2,"B",579),(3,"A",458),
                    (1,"A",531),(2,"C",143),(2,"B",554),(1,"C",496),(1,"A",732),(2,"C",416),(1,"B",167),(2,"A",584),(2,"B",579),(3,"A",458),
                    (1,"A",531),(2,"C",143),(2,"B",554),(1,"C",496),(1,"A",732),(2,"C",416),(1,"B",167),(2,"A",584),(2,"B",579),(3,"A",458),
                    (1,"A",831),(1,"C",143),(2,"C",654),(1,"C",486),(1,"A",832),(2,"C",517),(1,"B",173),(2,"A",364),(2,"B",589),(3,"A",457),
                    (1,"A",631),(1,"C",143),(2,"A",754),(1,"B",471),(1,"A",122),(2,"C",158),(1,"B",184),(2,"A",144),(2,"B",599),(3,"A",427),
                    (1,"A",699),(2,"C",143),(2,"B",654),(1,"A",272),(1,"A",824),(2,"C",169),(1,"B",195),(2,"A",223),(2,"B",219),(3,"A",415),
                    (1,"A",641),(3,"A",143),(2,"A",114),(3,"C",376),(2,"A",835),(1,"C",125),(2,"B",106),(2,"A",284),(1,"B",309),(3,"A",119),
                    (1,"A",186),(1,"A",235),(2,"B",351),(3,"A",133),(3,"A",857),(2,"A",215),(1,"A",152),(1,"A",209),(1,"C",215),(3,"C",206)]
                     ).toDF(["ID","X","Y"])
# df.show()

In [ ]:
# inputデータの件数確認
df.count()

250

In [ ]:
# シードと割合の設定
seed = 12
fractions = df.select("ID").distinct().withColumn("fraction", lit(0.5)).rdd.collectAsMap()
print(fractions)                                                       

{1: 0.5, 3: 0.5, 2: 0.5}


In [ ]:
sampled_df = df.stat.sampleBy("ID", fractions, seed)
# sampled_df.show()

In [ ]:
# サンプリング件数を確認
sampled_df.count()

128

In [ ]:
# ★サンプリング前のデータ
# 変数：IDごとの件数確認
df.groupBy("ID").count().sort("ID").show()

+---+-----+
| ID|count|
+---+-----+
|  1|  151|
|  2|   68|
|  3|   31|
+---+-----+



In [ ]:
# ★サンプリング後のデータ
# 変数：IDごとの件数確認
sampled_df.groupBy("ID").count().sort("ID").show()

+---+-----+
| ID|count|
+---+-----+
|  1|   68|
|  2|   44|
|  3|   16|
+---+-----+



## 2.層別サンプリング
sampleByKeyサンプル

https://stackoverflow.com/questions/43878019/pyspark-sampleby-using-multiple-columns


In [ ]:
# inputデータ作成
# 上で使ったデータを使用
# df.show()

In [ ]:
# 複数のkeyを使って、strata（層）を指定
#   ※ここでは、1つめ,2つめの変数を使用
# 各キーにサンプリング割合を割り当て
fractions = df.select("ID", "X").rdd.map(lambda x: (x[0],x[1])).distinct().map(lambda x: (x,0.5)).collectAsMap()
fractions

{(1, 'A'): 0.5,
 (1, 'B'): 0.5,
 (1, 'C'): 0.5,
 (2, 'A'): 0.5,
 (2, 'B'): 0.5,
 (2, 'C'): 0.5,
 (3, 'A'): 0.5,
 (3, 'C'): 0.5}

In [ ]:
# キーを設定
kb = df.rdd.keyBy(lambda x: (x[0],x[1]))
# kb.collect()

In [ ]:
# 新しくキー設定されたrddからサンプリング
# ※サンプルが値を返さなかった場合は、「ValueError：RDDisempty」エラーが発生
sampleddf = kb.sampleByKey(False,fractions,seed).map(lambda x: x[1]).toDF(df.columns)
# sampleddf.show()

In [ ]:
# サンプリング件数を確認
sampleddf.count()

124

In [ ]:
# ★サンプリング前のデータ
# グループごとのサンプリング件数を確認
df.groupBy("ID","X").count().sort("ID","X").show()

+---+---+-----+
| ID|  X|count|
+---+---+-----+
|  1|  A|  104|
|  1|  B|   35|
|  1|  C|   12|
|  2|  A|   22|
|  2|  B|   25|
|  2|  C|   21|
|  3|  A|   20|
|  3|  C|   11|
+---+---+-----+



In [ ]:
# ★サンプリング後のデータ
# グループごとのサンプリング件数を確認
sampleddf.groupBy("ID","X").count().sort("ID","X").show()

+---+---+-----+
| ID|  X|count|
+---+---+-----+
|  1|  A|   55|
|  1|  B|   15|
|  1|  C|    5|
|  2|  A|   14|
|  2|  B|   12|
|  2|  C|   12|
|  3|  A|    7|
|  3|  C|    4|
+---+---+-----+



## 3.変数重要度の算出

### Pandasの場合

In [44]:
# 必要なライブラリ読み込み
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [45]:
# 利用できるデータセットの一覧とその行数、列数を表示
dataset_list = sns.get_dataset_names()
for list_nm in dataset_list:
  print(list_nm, sns.load_dataset(list_nm).shape)

anagrams (20, 5)
anscombe (44, 3)
attention (60, 5)
brain_networks (923, 63)
car_crashes (51, 8)
diamonds (53940, 10)
dots (848, 5)
exercise (90, 6)
flights (144, 3)
fmri (1064, 5)
gammas (6000, 4)
geyser (272, 3)
iris (150, 5)
mpg (398, 9)
penguins (344, 7)
planets (1035, 6)
tips (244, 7)
titanic (891, 15)


In [46]:
# diamondsデータを使用
diamonds = sns.load_dataset("diamonds")
diamonds['target_flg'] = diamonds['cut'].apply(lambda x: 1 if x == 'Premium' else 0)

In [47]:
print('全件', diamonds.shape)
print('TGの件数', diamonds[diamonds["target_flg"] == 1].shape)

全件 (53940, 11)
TGの件数 (13791, 11)


In [52]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z,target_flg
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,0
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,1
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,0
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,1
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,0


In [49]:
def encodeing_label(data):
  le = LabelEncoder()
  le.fit(data)
  return le.transform(data)

In [50]:
def encoding_dataframe(df, drop_var):

  df_encd = df.copy()         #参照元のデータを置き換えないようにコピー
  #決定木に入れるためにカテゴリ変数を変換
  df_extracted = df.drop(drop_var, axis=1)
  df_label = df_extracted.select_dtypes(exclude=['number'])

  for i in df_label.columns:
    df_encd[i] = encodeing_label(df_encd[i])
  return df_encd

In [53]:
drop_var = ['x', 'y', 'z', 'target_flg','cut']
diamonds_encd = encoding_dataframe(diamonds, drop_var)
diamonds_encd.head()

,carat,cut,color,clarity,depth,table,price,x,y,z,target_flg
0,0.23,Ideal,1,3,61.5,55.0,326,3.95,3.98,2.43,0
1,0.21,Premium,1,2,59.8,61.0,326,3.89,3.84,2.31,1
2,0.23,Good,1,4,56.9,65.0,327,4.05,4.07,2.31,0
3,0.29,Premium,5,5,62.4,58.0,334,4.20,4.23,2.63,1
4,0.31,Good,6,3,63.3,58.0,335,4.34,4.35,2.75,0


In [54]:
def model_select_keep_var(df, drop_var):

  train_x, test_x, train_y, test_y = train_test_split(df.drop(drop_var, axis=1),
                                                       df['target_flg'],
                                                       random_state = 12345,
                                                       train_size = 0.8)
  tree = DecisionTreeClassifier(criterion = 'gini',
                                random_state = 12345
  ).fit(train_x, train_y)

  # 重要変数をDF化
  imp_df = pd.DataFrame(tree.feature_importances_, train_x.columns, columns=['freature_importances_'])
  # 上位50変数のみ
  imp_df = imp_df.sort_values('freature_importances_', ascending=False)[0:50]
  print('変数の重要度', imp_df)
  return list(imp_df.index)


In [55]:
keep_var = model_select_keep_var(diamonds_encd, drop_var)

変数の重要度          freature_importances_
table                 0.335951
price                 0.241432
depth                 0.219881
carat                 0.109338
color                 0.051723
clarity               0.041676


### Pysparkの場合

In [62]:
# 必要なライブラリ読み込み
import seaborn as sns
from pyspark.sql import functions as F
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.tree import DecisionTreeClassifier
# import pandas as pd

In [158]:
# diamondsデータを使用
diamonds = sqlContext.createDataFrame(sns.load_dataset("diamonds")).withColumn(
    "target_flg",
    F.when(F.col('cut') == 'Premium', 1)
    .otherwise(0)
)
diamonds.show(5)

+-----+-------+-----+-------+-----+-----+-----+----+----+----+----------+
|carat|    cut|color|clarity|depth|table|price|   x|   y|   z|target_flg|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+----------+
| 0.23|  Ideal|    E|    SI2| 61.5| 55.0|  326|3.95|3.98|2.43|         0|
| 0.21|Premium|    E|    SI1| 59.8| 61.0|  326|3.89|3.84|2.31|         1|
| 0.23|   Good|    E|    VS1| 56.9| 65.0|  327|4.05|4.07|2.31|         0|
| 0.29|Premium|    I|    VS2| 62.4| 58.0|  334| 4.2|4.23|2.63|         1|
| 0.31|   Good|    J|    SI2| 63.3| 58.0|  335|4.34|4.35|2.75|         0|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+----------+
only showing top 5 rows



In [69]:
print('全件', diamonds.count())
print('TGの件数', diamonds[diamonds["target_flg"] == 1].count())

全件 53940
TGの件数 13791


#### 決定木回帰  
http://mogile.web.fc2.com/spark/ml-classification-regression.html

In [93]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [97]:
# ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [98]:
# Load the data stored in LIBSVM format as a DataFrame.
data = spark.read.format("libsvm").load("/content/drive/MyDrive/data/data_mllib_sample_libsvm_data.txt")

In [100]:
data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [105]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)
featureIndexer

VectorIndexer_918864284b63

In [106]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [107]:
# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

In [108]:
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

In [109]:
# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

In [110]:
# Make predictions.
predictions = model.transform(testData)

In [111]:
# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(692,[95,96,97,12...|
|       0.0|  0.0|(692,[121,122,123...|
|       0.0|  0.0|(692,[123,124,125...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[125,126,127...|
+----------+-----+--------------------+
only showing top 5 rows



In [112]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.164399


In [152]:
treeModel = model.stages[1]
# summary only
print(treeModel)

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_671a3a1fb578) of depth 1 with 3 nodes


In [157]:
# 決定木の分岐条件
print(treeModel.toDebugString)

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_671a3a1fb578) of depth 1 with 3 nodes
  If (feature 434 <= 70.5)
   Predict: 0.0
  Else (feature 434 > 70.5)
   Predict: 1.0



In [156]:
# 変数の重要度
print(treeModel.featureImportances)

(692,[434],[1.0])


#### VectorIndexer
カテゴリカルフィーチャ列にインデックスを付ける機能！  
????  
https://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=vectorindexer#pyspark.ml.feature.VectorIndexer

In [227]:
from pyspark.ml.linalg import Vectors

In [233]:
df = spark.createDataFrame([
                            (Vectors.dense([-1.0, 0.0]),),
                            (Vectors.dense([0.0, 1.0]),),
                            (Vectors.dense([0.0, 2.0]),),
                            (Vectors.dense([-1.0, -2.0]),),
                            (Vectors.dense([-1.0, -3.2]),),
                            (Vectors.dense([0.0, -7.0]),)
                            ], ["a"])

In [234]:
# df = spark.createDataFrame([(Vectors.dense([-1.0, 0.0]),),
#     (Vectors.dense([0.0, 1.0]),), (Vectors.dense([0.0, 2.0]),)], ["a"])

In [235]:
df.show()

+-----------+
|          a|
+-----------+
| [-1.0,0.0]|
|  [0.0,1.0]|
|  [0.0,2.0]|
|[-1.0,-2.0]|
|[-1.0,-3.2]|
| [0.0,-7.0]|
+-----------+



In [238]:
indexer = VectorIndexer(maxCategories=2, inputCol="a")
indexer.setOutputCol("indexed")
indexer.getHandleInvalid()
model = indexer.fit(df)

In [239]:
df_trans = model.transform(df)
df_trans.show()

+-----------+----------+
|          a|   indexed|
+-----------+----------+
| [-1.0,0.0]| [1.0,0.0]|
|  [0.0,1.0]| [0.0,1.0]|
|  [0.0,2.0]| [0.0,2.0]|
|[-1.0,-2.0]|[1.0,-2.0]|
|[-1.0,-3.2]|[1.0,-3.2]|
| [0.0,-7.0]|[0.0,-7.0]|
+-----------+----------+



In [240]:
model.numFeatures

2

In [241]:
model.categoryMaps

{0: {0.0: 0, -1.0: 1}}

In [242]:
indexer.setParams(outputCol="test").fit(df).transform(df).collect()[1].test

DenseVector([0.0, 1.0])

In [243]:
params = {indexer.maxCategories: 3, indexer.outputCol: "vector"}
model2 = indexer.fit(df, params)
model2.transform(df).head().vector

DenseVector([1.0, 0.0])

In [248]:
dfWithInvalid = spark.createDataFrame([(Vectors.dense([3.0, 1.0]),)], ["a"])
indexer.getHandleInvalid()

'keep'

In [249]:
model3 = indexer.setHandleInvalid("skip").fit(df)
model3.transform(dfWithInvalid).count()

0

In [250]:
model4 = indexer.setParams(handleInvalid="keep", outputCol="indexed").fit(df)
model4.transform(dfWithInvalid).head().indexed

DenseVector([2.0, 1.0])

In [251]:
# よくわかんない？？？

#### VectorAssembler
複数列を1つの列にマージする機能！  
1列しか指定できない引数のときに、キー複数列用いたい時とかに使う？   
https://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=vectorassembler#pyspark.ml.feature.VectorAssembler

In [124]:
df = spark.createDataFrame([(1, 0, 3)], ["a", "b", "c"])

VectorAssembler_cb298ecf4ca8

In [130]:
df.show(5)

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  0|  3|
+---+---+---+



In [162]:
vecAssembler = VectorAssembler(outputCol="features")
vecAssembler.setInputCols(["a", "b", "c"])

VectorAssembler_211e8bb90557

In [150]:
df_trans = vecAssembler.transform(df)
df_trans.show(5)

+---+---+---+-------------+
|  a|  b|  c|     features|
+---+---+---+-------------+
|  1|  0|  3|[1.0,0.0,3.0]|
+---+---+---+-------------+



## 決定木の基本

In [ ]:
# pandasの場合の決定木
# DecisionTreeClassfier()

# imp_df = pd.Dataframe(tree.feature_importances_, train_x.columns, columns=["feature_importances_"])
# imp_df = imp_df.sort_values("feature_importances_", ascending=False)[0:50]
# # 先頭５０の重要変数を出力
# print("変数重要度の確認", imp_df)
